In [1]:
import os
os.chdir("../")

In [2]:
import glob
import pandas as pd
import time
import numpy as np
from scipy import stats

In [203]:
resultDir = 'results'
problem = 'setcover' # choices=['setcover', 'cauctions', 'facilities', 'indset']
sampling_Strategies = ['uniform5','depthK','depthK2'] # choices: uniform5, depthK, depthK2, depthK3
seeds = [0,1,2,3,4]

In [204]:
eval_files = glob.glob(f'{resultDir}/{problem}_*.csv')
eval_file = eval_files[-1]

df = pd.read_csv(eval_file)
df = pd.concat([df[df['type']=='small'], df[df['type']=='medium']])
df = df.astype({'nlps': float, 'nnodes' : float})

df_gcnns = df[df['policy'] != 'internal:relpscost']

In [205]:
def gmean_1shifted(x):
    return stats.mstats.gmean(x + 1) - 1

# 1. Means

In [206]:
dfgrouped_gmean = df_gcnns.groupby(['type','sampling_strategy'])[['nnodes', 'stime']].agg(gmean_1shifted)
dfgrouped_gmean

nnodes       stime
type   sampling_strategy                         
medium depthK             4190.855958  253.702280
       depthK2            3810.886626  232.361865
       uniform5           3715.530265  226.510969
small  depthK              203.614339   22.818759
       depthK2             201.301752   22.547455
       uniform5            200.481202   22.689888

In [207]:
dfgrouped_gmean.loc['small'] / dfgrouped_gmean.loc[('small', 'uniform5')]

,nnodes,stime
sampling_strategy,,
depthK,1.015628,1.005680
depthK2,1.004093,0.993723
uniform5,1.000000,1.000000


In [208]:
dfgrouped_gmean.loc[('medium')] / dfgrouped_gmean.loc[('medium', 'uniform5')]

,nnodes,stime
sampling_strategy,,
depthK,1.127929,1.120044
depthK2,1.025664,1.025831
uniform5,1.000000,1.000000


# 2. Std variances (per instance)

In [209]:
std_df = df_gcnns.groupby(['type','sampling_strategy','instance']).std() / df_gcnns.groupby(['type','sampling_strategy','instance']).mean()

In [210]:
std_df_final = std_df.groupby(['type','sampling_strategy'])[['nnodes','stime']].mean()
std_df_final

nnodes     stime
type   sampling_strategy                    
medium depthK             0.134848  0.115929
       depthK2            0.134719  0.116474
       uniform5           0.078641  0.070094
small  depthK             0.085575  0.047489
       depthK2            0.084428  0.049129
       uniform5           0.080582  0.040424

# 3. 计算Wins

In [212]:
df_uniform5 = df_gcnns[df_gcnns['sampling_strategy']=='uniform5']
df_depthK = df_gcnns[df_gcnns['sampling_strategy']=='depthK']
df_depthK2 = df_gcnns[df_gcnns['sampling_strategy']=='depthK2']
df_gcnns['Wins'] = 0

/opt/anaconda3/envs/learn2branch/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [193]:
for i in range(0,len(df_uniform5)):
    uniform5_row = df_uniform5.iloc[i]
    depthK_row = df_gcnns[
        (df_gcnns['sampling_strategy'] == 'depthK') &
        (df_gcnns['policy'] == uniform5_row.policy) &
        (df_gcnns['seed'] == uniform5_row.seed) &
        (df_gcnns['type'] == uniform5_row.type) &
        (df_gcnns['instance'] == uniform5_row.instance)
    ]
    depthK2_row = df.loc[
        (df_gcnns['sampling_strategy'] == 'depthK2') &
        (df_gcnns['policy'] == uniform5_row.policy) &
        (df_gcnns['seed'] == uniform5_row.seed) &
        (df_gcnns['type'] == uniform5_row.type) &
        (df_gcnns['instance'] == uniform5_row.instance)
    ]
    winner = np.argmin([uniform5_row.stime, depthK_row.stime.iloc[0], depthK2_row.stime.iloc[0]])
    winner = ['uniform5', 'depthK', 'depthK2'][winner]
    df_gcnns.loc[ (df_gcnns['sampling_strategy'] == winner) &
            (df_gcnns['policy'] == uniform5_row.policy) &
            (df_gcnns['seed'] == uniform5_row.seed) &
            (df_gcnns['type'] == uniform5_row.type) &
            (df_gcnns['instance'] == uniform5_row.instance), 'Wins'] += 1

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [183]:
df_gcnns.groupby(['type', 'sampling_strategy']).sum()

seed    nnodes      nlps     stime       gap  \
type   sampling_strategy                                                 
medium depthK              200   64247.0   84807.0  36220.05  0.000000   
       depthK2             200   63639.0   83625.0  35724.68  0.000000   
       uniform5            200   64226.0   84277.0  36416.79  0.000000   
small  depthK              200  126692.0  152892.0  25820.21  0.013059   
       depthK2             200  124882.0  153451.0  26464.66  0.017420   
       uniform5            200  118822.0  145599.0  26367.50  0.019083   

                          ndomchgs  ncutoffs      walltime      proctime  Wins  
type   sampling_strategy                                                        
medium depthK                    0         0  36708.903560  36727.453125    25  
       depthK2                   0         0  36222.645429  36231.656250    42  
       uniform5                  0         0  36908.882507  36924.421875    33  
small  depthK                    0         0  26368.496764  26428.218750    39  
       depthK2                   0         0  27020.186683  27078.796875    36  
       uniform5                  0         0  26912.304422  26978.437500    25